In [1]:
import cv2
import os
import numpy as np
#將所有圖片調成長邊為160且短邊等比例縮放的區塊
def resize_all_images(input_dir,output_dir,set_size=160):
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg','png','.jpeg')):
            img_path=os.path.join(input_dir,filename)
            img=cv2.imread(img_path)
            h,w=img.shape[:2]
            #開始縮放，固定長邊為set_size
            scale=set_size/max(h,w)
            nw,nh=int(scale* w),int(scale*h)
            resized_img=cv2.resize(img,(nw,nh),interpolation=cv2.INTER_LINEAR)
            #建立背景
            padding_img=np.zeros((set_size,set_size,3),dtype=np.uint8)
            #開始padding
            top= (set_size-nh)//2
            left= (set_size-nw)//2
            #將縮放後的圖片放入背景
            padding_img[top:top+nh,left:left+nw,:]=resized_img
            #保存
            output_path=os.path.join(output_dir,filename)
            cv2.imwrite(output_path,padding_img)
            print(f'processed {filename}')
resize_all_images(r"D:\addingphoto",r"D:\addingphoto160",set_size=160)

processed IMG_20231111_110753.jpg
processed IMG_20231111_114254.jpg
processed IMG_20231111_120230.jpg
processed IMG_20231111_124210.jpg
processed IMG_20231111_124516.jpg
processed IMG_20231111_132811.jpg
processed IMG_20231111_135921.jpg
processed IMG_20231113_104743.jpg
processed IMG_20231113_104837.jpg
processed IMG_20231114_084049.jpg
processed IMG_20231114_184732.jpg
processed IMG_20231119_130326.jpg
processed IMG_20231119_131827.jpg
processed IMG_20231119_135403.jpg
processed IMG_20231119_154207.jpg
processed IMG_20231119_155406.jpg
processed IMG_20231119_155700.jpg
processed IMG_20231128_181357.jpg
processed IMG_20231128_181404.jpg
processed IMG_20231128_181407.jpg
processed IMG_20231201_084801.jpg
processed IMG_20231202_182516.jpg
processed IMG_20231206_165905.jpg
processed IMG_20231207_130829.jpg
processed IMG_20231208_165826.jpg
processed IMG_20231211_083210.jpg
processed IMG_20231213_125925.jpg
processed IMG_20231214_170416.jpg
processed IMG_20231216_095353.jpg
processed IMG_

In [3]:
import cv2
import os
import numpy as np
oid=r"D:\addingphoto"
rid=r"D:\addingphoto160"
old=r"D:\addingphotolabels"
rld=r"D:\addingphotolabels160"
#固定長邊尺寸
set_size=160
def resize_labels(image_name):
    oip=os.path.join(oid,image_name)
    img=cv2.imread(oip)
    h,w=img.shape[:2]
    #計算縮放比與填充
    scale=set_size/max(h,w)
    nw,nh=int(scale* w),int(scale*h)
    top= (set_size-nh)//2
    left= (set_size-nw)//2
    #處理對應標籤檔
    label_name=os.path.splitext(image_name)[0]+".txt"
    olp=os.path.join(old,label_name)
    rlp=os.path.join(rld,label_name)
    #如果標籤檔不存在，跳過
    if not os.path.exists(olp):
        print(f'label file not found:{olp}')
        return
    #讀取原始標籤檔
    with open(olp,'r') as f:
        lines=f.readlines()
    #更新標籤
    updated_label=[]
    for line in lines:
        parts=line.strip().split()
        if len(parts)<5:
            continue
        class_id,x_center,y_center,width,height=map(float,parts)
        #相對比例到絕對座標(原始圖片)
        x_center_abs=x_center*w
        y_center_abs=y_center*h
        width_abs=width*w
        height_abs=height*h
        #縮放和偏移
        x_center_resize=x_center_abs*scale+left
        y_center_resize=y_center_abs*scale+top
        width_resize=width_abs*scale
        height_resize=height_abs*scale
        #絕對座標到相對比例(縮放後圖像)
        x_center_new=x_center_resize/set_size
        y_center_new=y_center_resize/set_size
        width_new=width_resize/set_size
        height_new=height_resize/set_size
        #添加到新標籤列
        updated_label.append(f'{class_id},{x_center_new:.6f},{y_center_new:.6f},{width_new:.6f},{height_new:.6f}')
    #保存新標籤檔
    with open(rlp,'w') as f:
        f.write("\n".join(updated_label))
    print(f'processed label for {image_name}')
#對所有圖片處理
for image_filename in os.listdir(oid):
    if image_filename.endswith(('.jpg','png','.jpeg')):
        resize_labels(image_filename)

processed label for IMG_20231111_110753.jpg
processed label for IMG_20231111_114254.jpg
processed label for IMG_20231111_120230.jpg
processed label for IMG_20231111_124210.jpg
processed label for IMG_20231111_124516.jpg
processed label for IMG_20231111_132811.jpg
processed label for IMG_20231111_135921.jpg
processed label for IMG_20231113_104743.jpg
processed label for IMG_20231113_104837.jpg
processed label for IMG_20231114_084049.jpg
processed label for IMG_20231114_184732.jpg
processed label for IMG_20231119_130326.jpg
processed label for IMG_20231119_131827.jpg
processed label for IMG_20231119_135403.jpg
processed label for IMG_20231119_154207.jpg
processed label for IMG_20231119_155406.jpg
processed label for IMG_20231119_155700.jpg
processed label for IMG_20231128_181357.jpg
processed label for IMG_20231128_181404.jpg
processed label for IMG_20231128_181407.jpg
processed label for IMG_20231201_084801.jpg
processed label for IMG_20231202_182516.jpg
processed label for IMG_20231206

In [2]:
import os
label_dir=r"D:\addingphotolabels160"
def convert_label_format(file_path):
    with open(file_path,'r') as f:
        lines=f.readlines()
    new_lines=[]
    for line in lines:
        #解析原標籤格式
        parts=line.strip().split(',')
        if len(parts)>=5:
            cls=int(float(parts[0]))#id=>int
            bbox=[float(x)  for x in parts[1:5]]
            new_lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in bbox)}\n")
    with open(file_path,'w') as f:#寫回舊格式
        f.writelines(new_lines)
for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):
        file_path=os.path.join(label_dir,label_file)
        convert_label_format(file_path)
        print(f'converted: {file_path}')
print('mission is over')

converted: D:\addingphotolabels160\IMG_20231111_110753.txt
converted: D:\addingphotolabels160\IMG_20231111_114254.txt
converted: D:\addingphotolabels160\IMG_20231111_120230.txt
converted: D:\addingphotolabels160\IMG_20231111_124210.txt
converted: D:\addingphotolabels160\IMG_20231111_124516.txt
converted: D:\addingphotolabels160\IMG_20231111_132811.txt
converted: D:\addingphotolabels160\IMG_20231111_135921.txt
converted: D:\addingphotolabels160\IMG_20231113_104743.txt
converted: D:\addingphotolabels160\IMG_20231113_104837.txt
converted: D:\addingphotolabels160\IMG_20231114_084049.txt
converted: D:\addingphotolabels160\IMG_20231114_184732.txt
converted: D:\addingphotolabels160\IMG_20231119_130326.txt
converted: D:\addingphotolabels160\IMG_20231119_131827.txt
converted: D:\addingphotolabels160\IMG_20231119_135403.txt
converted: D:\addingphotolabels160\IMG_20231119_154207.txt
converted: D:\addingphotolabels160\IMG_20231119_155406.txt
converted: D:\addingphotolabels160\IMG_20231119_155700.t

In [4]:
import os

# 圖片和標籤的目錄
image_dir = r"C:\Users\st030\train2017"
label_dir = r"C:\Users\st030\labels"

# 儲存缺少標籤的圖片檔
missing_labels = []

for image_file in os.listdir(image_dir):
    if image_file.endswith(".jpg"):
        label_file = os.path.join(label_dir, image_file.replace(".jpg", ".txt"))
        image_path = os.path.join(image_dir, image_file)
        
        # 如果對應的標籤檔不存在
        if not os.path.exists(label_file):
            missing_labels.append(image_file)
            # 刪除沒有對應標籤的圖片檔
            os.remove(image_path)
            print(f"Deleted image without label: {image_file}")

# 最後輸出檢查結果
if missing_labels:
    print(f"Missing labels for {len(missing_labels)} images. These images have been deleted.")
else:
    print("All images have corresponding labels.")

Deleted image without label: 000000000078.jpg
Deleted image without label: 000000000208.jpg
Deleted image without label: 000000000250.jpg
Deleted image without label: 000000000491.jpg
Deleted image without label: 000000000508.jpg
Deleted image without label: 000000000626.jpg
Deleted image without label: 000000000801.jpg
Deleted image without label: 000000000873.jpg
Deleted image without label: 000000001084.jpg
Deleted image without label: 000000001111.jpg
Deleted image without label: 000000001340.jpg
Deleted image without label: 000000001392.jpg
Deleted image without label: 000000001472.jpg
Deleted image without label: 000000001497.jpg
Deleted image without label: 000000001505.jpg
Deleted image without label: 000000001647.jpg
Deleted image without label: 000000001674.jpg
Deleted image without label: 000000001681.jpg
Deleted image without label: 000000001852.jpg
Deleted image without label: 000000001997.jpg
Deleted image without label: 000000002024.jpg
Deleted image without label: 00000

In [ ]:
import os

# 標籤檔目錄
label_dir = r"C:\Users\st030\coco\train2017\labels"

# 定義 class_id 的上限值
class_id_threshold = 79

# 逐一處理標籤檔
for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        label_path = os.path.join(label_dir, label_file)
        
        # 新的標籤資料儲存列表
        valid_labels = []
        
        # 讀取標籤檔內容
        with open(label_path, 'r') as file:
            lines = file.readlines()
        
        # 篩選出 class_id 小於等於 79 的行
        for line in lines:
            parts = line.strip().split()
            if parts:
                class_id = int(parts[0])
                if class_id <= class_id_threshold:
                    valid_labels.append(line)
                else:
                    print(f"Removed line from {label_file}: {line.strip()}")
        
        # 將有效的標籤內容重新寫回檔案
        with open(label_path, 'w') as file:
            file.writelines(valid_labels)

print("Finished processing all label files.")

Removed line from 000000000030.txt: 85 0.501188 0.558492 0.260000 0.305078
Removed line from 000000000064.txt: 84 0.420609 0.184914 0.241094 0.237984
Removed line from 000000000089.txt: 83 0.815477 0.670515 0.141328 0.047438
Removed line from 000000000089.txt: 83 0.865914 0.721938 0.178203 0.052656
Removed line from 000000000089.txt: 83 0.900883 0.793390 0.198234 0.078313
Removed line from 000000000127.txt: 83 0.502742 0.500578 0.201672 0.144562
Removed line from 000000000133.txt: 87 0.858305 0.180141 0.074922 0.044875
Removed line from 000000000138.txt: 81 0.122172 0.402703 0.192281 0.384563
Removed line from 000000000138.txt: 84 0.546672 0.134141 0.107031 0.107031
Removed line from 000000000138.txt: 80 0.877086 0.409281 0.201766 0.083625
Removed line from 000000000138.txt: 85 0.367500 0.172765 0.052562 0.073688
Removed line from 000000000164.txt: 81 0.745406 0.500140 0.151125 0.215687
Removed line from 000000000165.txt: 86 0.341414 0.501883 0.467797 0.447047
Removed line from 0000000